# Session 9 - SOLID Design Principles and Dependency Injection

We've covered many different features of C# as a language, gotten into a bit of the .NET Core and .NET Framework features, but we have not yet talked about practical class design practices.  In this session, we'll learn about SOLID principles of class design and how to take advantage of dependency injection for loose coupling between objects.

The goals of introducing these design patterns and practices are to improve maintainability of our code, minimumze unintended performance issues, and create a design system that is easy for other developers to learn.

## SOLID - An introduction

SOLID is an acronym for 5 design principles introduced by [Robert C. Martin in his 2000 paper 'Design Principles and Design Patterns'](https://web.archive.org/web/20150906155800/http://www.objectmentor.com/resources/articles/Principles_and_Patterns.pdf) and the term [SOLID was coined by Michael Feathers later](https://en.wikipedia.org/wiki/SOLID).

There are some developers who are quite dogmatic in their approach to building systems and require strict adherence to these principles. They sometimes go as far as to utilize static analysis tools to verify that the rules are not broken.  For many, these are a series of recommendations that are chosen to be used and sometimes chosen to be violated in order to produce good software.

As always, I recommend that you adhere to the number one feature requirement of software development:  Ship usable software.

## S: Single Responsibility Principle (SRP)

    A class should only have a single responsibility, that is, 
		only changes to one part of the software's specification
		 should be able to affect the specification of the class.
    
Let's simplify:  a class should only have one reason to be changed or rewritten.  Conceptually, this means that you should have many smaller class objects that you can use to reference each other to perform a task.

Consider a `Logger` object that will be used to track the running state of an application as well as to report errors for later analysis.  What tasks should it perform?

- Create a log file
- Name and rotate the log files
- Write log messages to a log file
- Format the log messages
- Close the log file

In [ ]:
class TheLogger : IDisposable {
    
    public void CreateNewFile(string filename) {
        // actions to create log files
    }
    
    public string NameNewLogFile() {
        return DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
    }
    
    public void LogMessage(string message) {
        // Add message to current log file
        var formattedMessage = Format(message);
    }
    
    public string Format(string rawMessage) {
        return $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
    }
    
    public void CloseLogFile() {
        // close the log file
    }
    
    public void Dispose() {
        CloseLogFile();
    }
    
}

var logger = new TheLogger();
display(logger.Format("Error detected!"));
logger.Dispose();

20211025_134823 - Error detected!

That's a lot of responsibilities of a logger.  What happens when we want to change the format of our messages?  A new requirement has been received that indicates we should centralize our logging on a web server using REST, how do we update our logger?

Let's simplify our `Logger` operations by breaking it out into several classes that each own a part of the message logging operation:

In [ ]:
class FileLogger : IDisposable {
    
    string _Filename;
    
    public void CreateNewFile(string filename) {
        // Create the log file
        _Filename = filename;
        display($"Created log file {filename}");
    }
    
    public void WriteMessage(string message) {
        // write message to log file
        display("Wrote message to disk");
    }
    
    public void Dispose() {
        // clean up and close the log file
        display($"Cleaned up and closed log file {_Filename}");
    }
    
}

class LoggerFilenameManager {
    
    public string NameNewLogFile() {
        var filename = DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
        display(filename);
        return filename;
    }
    
}

class LoggerFormatter {
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
        display(msg);
        return msg;
    }
    
}

// Compose these objects into a single LoggerManager object that handles the operations for ther rest of our application
class LoggerManager : IDisposable {
    
    FileLogger _Logger = new FileLogger();
    LoggerFilenameManager _FilenameManager = new LoggerFilenameManager();
    LoggerFormatter _Formatter = new LoggerFormatter();
    
    public LoggerManager() {
        var filename = _FilenameManager.NameNewLogFile();
        _Logger.CreateNewFile(filename);
    }
    
    public void LogMessage(string message) {
        var newMessage = _Formatter.FormatLogMessage(message);
        _Logger.WriteMessage(newMessage);
    }
    
    public void Dispose() {
        _Logger.Dispose();
    }
    
}

var logger = new LoggerManager();
logger.LogMessage("Error detected");
logger.Dispose();

20211025_13.log

Created log file 20211025_13.log

20211025_135709 - Error detected

Wrote message to disk

Cleaned up and closed log file 20211025_13.log

That's MUCH better.  Consider the various modification points for our logger and how we can now maintain this system of objects:

- Log record format changes can be made inside the `LoggerFormatter` class
- Log filename changes can be made in the `LoggerFilenameManager` class
- Log destination changes can be made by replacing the `Filelogger` in our `LoggerManager` class

## O: The Open / Closed Principle (OCP)

    Software entities (classes, modules, functions, etc.) 
		should be open for extension, but closed for modification
    
Implementing this principle will begin to dramatically change the way that you think and interact with your systems by encouraging new features to be created **ONLY** by creating new code.  

Revisiting our Logger example, let's consider what happens if we want to introduce a feature that formats messages differently based on their severity.  Clearly, we need to change the LoggerFormatter.  Let's take a look:

In [ ]:
class FileLogger : IDisposable {
    
    string _Filename;
    
    public void CreateNewFile(string filename) {
        // Create the log file
        _Filename = filename;
        display($"Created log file {filename}");
    }
    
    public void WriteMessage(string message) {
        // write message to log file
        display("Wrote message to disk");
    }
    
    public void Dispose() {
        // clean up and close the log file
        display($"Cleaned up and closed log file {_Filename}");
    }
    
}

class LoggerFilenameManager {
    
    public string NameNewLogFile() {
        var filename = DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
        display(filename);
        return filename;
    }
    
}

interface ILoggerFormatter {
    string FormatLogMessage(string rawMessage);
}

class NormalLoggerFormatter : ILoggerFormatter {
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
        display(msg);
        return msg;
    }
    
}

class AlertLoggerFormatter : ILoggerFormatter {
    
	public string FormatLogMessage(string rawMessage) {
			var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} " +
			$" - ALERT! {rawMessage}";
			display(msg);
			return msg;
	}
	
}

// Compose these objects into a single LoggerManager object that handles the operations for ther rest of our application
class LoggerManager : IDisposable {
    
    FileLogger _Logger = new FileLogger();
    LoggerFilenameManager _FilenameManager = new LoggerFilenameManager();
    
    public LoggerManager() {
        var filename = _FilenameManager.NameNewLogFile();
        _Logger.CreateNewFile(filename);
    }
    
    public void LogMessage(string message) {
        LogMessage(message, new NormalLoggerFormatter());
    }
    
    public void LogMessage(string message, ILoggerFormatter formatter) {
        var newMessage = formatter.FormatLogMessage(message);
        _Logger.WriteMessage(newMessage);
    }
    
    public void Dispose() {
        _Logger.Dispose();
    }
    
}

var logger = new LoggerManager();
logger.LogMessage("Something REALLY bad happened", new AlertLoggerFormatter());
logger.Dispose();

20211025_14.log

Created log file 20211025_14.log

20211025_140756  - ALERT! Something REALLY bad happened

Wrote message to disk

Cleaned up and closed log file 20211025_14.log

With the introduction of the `ILoggerFormatter` interface and acceptance through the LogMessage method, we can add new formats when exceptions occur or other scenarios that we want to be able to log differently in our code:

In [ ]:
class ExceptionLogFormatter : ILoggerFormatter {

    Exception _Exception;
    
    public ExceptionLogFormatter(Exception ex) {
        _Exception = ex;
    }
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - Exception of type {_Exception.GetType().Name} with message '{_Exception.Message}' logged: {rawMessage}";
        display(msg);
        return msg;
    }
}

var logger = new LoggerManager();
logger.LogMessage("Error detected", 
	new ExceptionLogFormatter(
		new Exception("Everything is on fire!")
	));
logger.Dispose();

20211025_14.log

Created log file 20211025_14.log

20211025_140850 - Exception of type Exception with message 'Everything is on fire!' logged: Error detected

Wrote message to disk

Cleaned up and closed log file 20211025_14.log

We can now **inject** the new features into our `LoggerManager` to enhance the formatting of our messages by simply creating new classes that implement the `ILoggerFormatter`.  This technique allows for the writing of log messages and the maintenance of our LoggerManager to remain closed to modification, but open for injecting new capabilities. 

## L: Liskov Substitution Principle (LSP)

    "Objects in a program should be replaceable 
		with instances of their subtypes without 
		altering the correctness of that program." 

[Originally defined](https://en.wikipedia.org/wiki/Liskov_substitution_principle) by [Turing Award winner and MIT professor Barbara Liskov](https://en.wikipedia.org/wiki/Barbara_Liskov) in her 1987 conference keynote address titled _Data abstraction and hierarchy_.

This can be simplified to mean that classes that reference base classes can use derived classes without needing to be aware that it is a subclass.  Let's work on that `FileLogger` implementation and how we can swap that out for a potential remote logging solution using REST.

In [ ]:
abstract class LogSink : IDisposable {
    
    public abstract void WriteMessage(string message);
    
    public virtual void Dispose() { }
    
}

class FileLogger : LogSink //, IDisposable 
{
    
    string _Filename;

    // This field and the constructor were added to complete the LSP implementation
    LoggerFilenameManager _FilenameManager = new LoggerFilenameManager();

    public FileLogger() {
        _Filename = _FilenameManager.NameNewLogFile();
        CreateNewFile(_Filename);
    }
    // end LSP implementation
    
    public void CreateNewFile(string filename) {
        // Create the log file
        _Filename = filename;
        display($"Created log file {filename}");
    }
    
    public override void WriteMessage(string message) {
        // write message to log file
        display("Wrote message to disk");
    }
    
    public override void Dispose() {
        // clean up and close the log file
        display($"Cleaned up and closed log file {_Filename}");
    }
    
}

class LoggerFilenameManager {
    
    public string NameNewLogFile() {
        var filename = DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
        display(filename);
        return filename;
    }
    
}

interface ILoggerFormatter {
    string FormatLogMessage(string rawMessage);
}

class NormalLoggerFormatter : ILoggerFormatter {
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
        display(msg);
        return msg;
    }
    
}

// Compose these objects into a single LoggerManager object that handles the operations for ther rest of our application
class LoggerManager : IDisposable {
    
    LogSink _Logger = new FileLogger();
    // LoggerFilenameManager _FilenameManager = new LoggerFilenameManager();
    
    public LoggerManager() {
//        var filename = _FilenameManager.NameNewLogFile();
//        _Logger.CreateNewFile(filename);
    }
    
    public void LogMessage(string message) {
        LogMessage(message, new NormalLoggerFormatter());
    }
    
    public void LogMessage(string message, ILoggerFormatter formatter) {
        var newMessage = formatter.FormatLogMessage(message);
        _Logger.WriteMessage(newMessage);
    }
    
    public void Dispose() {
        _Logger.Dispose();
    }
    
}

var logger = new LoggerManager();
logger.LogMessage("Error detected");
logger.Dispose();

20211025_14.log

Created log file 20211025_14.log

20211025_141625 - Error detected

Wrote message to disk

Cleaned up and closed log file 20211025_14.log

## I: Interface Segregation Principle (ISP)

    Many client-specific interfaces are better than one general-purpose interface.
    
This principle encourages you to create systems that allow classes to only depend on those objects that deliver _JUST_ the functionality that it needs.
    
This principle is responsible for the proliferation of many little interfaces in much of the .NET ecosystem.  Smaller, focused interfaces allow for more flexibility in delivering code.  However, it is possible to get carried away and create TOO many interfaces.

For this system, we could define a small collection of interfaces that comprise the various components of the `LoggerManager`:

In [ ]:
interface ILoggerFormatter {
    string FormatLogMessage(string rawMessage);
}

interface IFilenameManager {
    string NameNewLogFile();
}

// We could now create a single object that implements these two methods for our system:

public class FileHandler : ILoggerFormatter, IFilenameManager {
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
        display(msg);
        return msg;
    }

    public string NameNewLogFile() {
        var filename = DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
        display(filename);
        return filename;
    }
    
}

## D: Dependency Inversion Principle (DIP)

    One should depend upon abstractions, [not] concretions.
    
Quite simply:  don't depend directly on another class.  When you depend on the implementation, name, and shape of another class you are **tightly coupled** to that class and changes to it will have an effect on your classes.

You can achieve more **loose coupling** by referring only to the abstraction and interfaces that your other classes adhere to in order to make your applications more maintainable without changing significant blocks of code.

How can we re-implement our logger system so that it only depends on the interfaces and abstract classes that we defined previously? 

In [ ]:
// The original Logger

class TheLogger : IDisposable {
    
    public void CreateNewFile(string filename) {
        // actions to create log files
    }
    
    public string NameNewLogFile() {
        return DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
    }
    
    public void LogMessage(string message) {
        // Add message to current log file
        var formattedMessage = Format(message);
    }
    
    public string Format(string rawMessage) {
        return $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
    }
    
    public void CloseLogFile() {
        // close the log file
    }
    
    public void Dispose() {
        CloseLogFile();
    }
    
}

// Re-implemented with abstractions and interfaces for maintenance
var logger = new TheLogger();
display(logger.Format("Error detected!"));
logger.Dispose();

In [ ]:
abstract class LogSink : IDisposable {
    
    public abstract void WriteMessage(string message);
    
    public virtual void Dispose() { }
    
}

interface IFilenameManager {
    string NameNewLogFile();
}

class FileLogger : LogSink {
    
    string _Filename;

    // Now referencing the interface as a type instead of the FilenameManager class
    IFilenameManager _FilenameManager = new LoggerFilenameManager();

    public FileLogger() {
        _Filename = _FilenameManager.NameNewLogFile();
        CreateNewFile(_Filename);
    }
    
    public void CreateNewFile(string filename) {
        // Create the log file
        _Filename = filename;
        display($"Created log file {filename}");
    }
    
    public override void WriteMessage(string message) {
        // write message to log file
        display("Wrote message to disk");
    }
    
    public override void Dispose() {
        // clean up and close the log file
        display($"Cleaned up and closed log file {_Filename}");
    }
    
}

class LoggerFilenameManager : IFilenameManager {
    
	public string NameNewLogFile() {
			var filename = DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
			display(filename);
			return filename;
	}
	
}

class AmericanSillyDateFormatFilenameManager : IFilenameManager {
    
	public string NameNewLogFile() {
			var filename = DateTime.UtcNow.ToString("MMddyyyy_HH") + ".log";
			display(filename);
			return filename;
	}
	
}



interface ILoggerFormatter {
    string FormatLogMessage(string rawMessage);
}

class NormalLoggerFormatter : ILoggerFormatter {
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
        display(msg);
        return msg;
    }
    
}

// Compose these objects into a single LoggerManager object that handles the operations for ther rest of our application
class LoggerManager : IDisposable {
    
    // Referencing the LogSink abstract class so that we can change our abstract interaction with LogSinks later
    LogSink _Logger = new FileLogger();
    
    public LoggerManager() {
    }
    
    public void LogMessage(string message) {
        LogMessage(message, new NormalLoggerFormatter());
    }
    
    
    // Referencing the ILoggerFormatter so that we can inject the desired formatter
    public void LogMessage(string message, ILoggerFormatter formatter) {
        var newMessage = formatter.FormatLogMessage(message);
        _Logger.WriteMessage(newMessage);
    }
    
    public void Dispose() {
        _Logger.Dispose();
    }
    
}

var logger = new LoggerManager();
logger.LogMessage("Error detected");
logger.Dispose();

20211025_14.log

Created log file 20211025_14.log

20211025_143723 - Error detected

Wrote message to disk

Cleaned up and closed log file 20211025_14.log

## Dependency Injection

Now that we understand the SOLID principles, let's take the dependency inversion principle and use that throughout our system using the **Dependency Injection** architecture pattern.  This pattern enforces a simple statement:  

    'New is glue'
    
Don't create your dependencies inside of your objects, and instead prefer them to be **injected** by an outside manager.  This outside manager is typically another class that is referred to as a **Service Locator** or a **Container**.  You'll see this technique referred to as **Inversion of Control** as you are giving control of the creation of the dependencies to another object that will be maintaining them for you.

Dependency Injection comes in several forms:

- Constructor Injection - the most common and requires that dependencies be provided in the constructor of a class
- Property Injection - allows dependencies to be optionally set through properties on a class
- Method Injection - dependencies are provided only on the methods where they are interacted with at the time they are executed

Let's refactor our `LoggerManager` further using the Dependency Injection pattern:

In [ ]:
abstract class LogSink : IDisposable {
    
    public abstract void WriteMessage(string message);
    
    public virtual void Dispose() { }
    
}

interface IFilenameManager {
    string NameNewLogFile();
}

class FileLogger : LogSink {
    
    string _Filename;

    // Now referencing the interface as a type instead of the FilenameManager class
    IFilenameManager _FilenameManager;

    public FileLogger(IFilenameManager filenameManager) {
        _FilenameManager = filenameManager;
        _Filename = _FilenameManager.NameNewLogFile();
        CreateNewFile(_Filename);
    }
    
    public void CreateNewFile(string filename) {
        // Create the log file
        _Filename = filename;
        display($"Created log file {filename}");
    }
    
    public override void WriteMessage(string message) {
        // write message to log file
        display("Wrote message to disk");
    }
    
    public override void Dispose() {
        // clean up and close the log file
        display($"Cleaned up and closed log file {_Filename}");
    }
    
}

class LoggerFilenameManager : IFilenameManager {
    
    public string NameNewLogFile() {
        var filename = DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
        display(filename);
        return filename;
    }
    
}

interface ILoggerFormatter {
    string FormatLogMessage(string rawMessage);
}

class NormalLoggerFormatter : ILoggerFormatter {
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
        display(msg);
        return msg;
    }
    
}

// Compose these objects into a single LoggerManager object that handles the operations for ther rest of our application
class LoggerManager : IDisposable {
    
    // Referencing the LogSink abstract class so that we can change our abstract interaction with LogSinks later
    LogSink _Logger;
    
    public LoggerManager(LogSink sink) {
        _Logger = sink;
    }
    
    public void LogMessage(string message) {
        LogMessage(message, new NormalLoggerFormatter());
    }
    
    
    // Referencing the ILoggerFormatter so that we can inject the desired formatter
    public void LogMessage(string message, ILoggerFormatter formatter) {
        var newMessage = formatter.FormatLogMessage(message);
        _Logger.WriteMessage(newMessage);
    }
    
    public void Dispose() {
        _Logger.Dispose();
    }
    
}

var fileNameManager = new LoggerFilenameManager();
var fileSink = new FileLogger(fileNameManager);
var logger = new LoggerManager(fileSink);
logger.LogMessage("Error detected");
logger.Dispose();

20211025_14.log

Created log file 20211025_14.log

20211025_144631 - Error detected

Wrote message to disk

Cleaned up and closed log file 20211025_14.log

In [ ]:
class FritzContainer {
    
    private static readonly Dictionary<Type, Type> _Mapping = new Dictionary<Type,Type>();
    
    public static void RegisterMapping<I, T>() where T: new()  {
        _Mapping.Add(typeof(I), typeof(T));
    }
    
    public static T Resolve<T>() {
        return new _Mapping[T]();
    }
    
}

var container = new FritzContainer();
container.RegisterMapping<IFilenameManager, LoggerFilenameManager>();
var mgr = container.Resolve<IFilenameManager>();
display(mgr.GetType())


Error: (17,11): error CS0176: Member 'FritzContainer.Resolve<IFilenameManager>()' cannot be accessed with an instance reference; qualify it with a type name instead
(16,1): error CS0176: Member 'FritzContainer.RegisterMapping<IFilenameManager, LoggerFilenameManager>()' cannot be accessed with an instance reference; qualify it with a type name instead
(10,20): error CS0246: The type or namespace name '_Mapping' could not be found (are you missing a using directive or an assembly reference?)
(10,29): error CS0119: 'T' is a type, which is not valid in the given context